In [1]:
import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import umap
from sklearn.cluster import MiniBatchKMeans,AgglomerativeClustering
from sklearn.decomposition import PCA
from util.plot import plotembeddingbyCT,plotembeddingbyCT_contrast
import cv2
import sklearn.metrics 

from util.plot_new import *

In [2]:
radius=48
datadir='/media/xinyi/dcis2idc/data'
with open(os.path.join(datadir,'processed','train_cnnvae_names'), 'rb') as input:
    allImgNames=pickle.load(input)
name='exp0'
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name
modelsavepath='/media/xinyi/dcis2idc/models/cnnvae'+name
plottype='umap'
sampledir=plotsavepath
savedir=os.path.join(sampledir,'embedding_'+plottype)
clustersavedir=os.path.join(sampledir,'cluster')

In [3]:
with open(os.path.join(datadir,'processed','train_cnnvae_coord'), 'rb') as output:
    coordlist=pickle.load(output)

In [ ]:
#get cellID -- label not row index
cellIDlist={}
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)
for sidx in range(uniquenames.size):
    s=uniquenames[sidx]
    posPath=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'spatial_positioning',s.split('_')[-1] +'.csv')
    pos=pd.read_csv(posPath)
    
    includedPos=coordlist[allImgNames==s]
    cellIDlist[s]=np.zeros(includedPos.shape[0])
    for i in range(includedPos.shape[0]):
        cellIDlist[s][i]=pos.to_numpy()[np.logical_and(pos.iloc[:,2]==includedPos[i,0],pos.iloc[:,3]==includedPos[i,1]),1]
    assert np.sum(cellIDlist[s]==0)==0

In [26]:
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels'), 'wb') as output:
    pickle.dump(cellIDlist,output, pickle.HIGHEST_PROTOCOL)

In [4]:
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels'), 'rb') as output:
    cellIDlist=pickle.load(output)

## cluster vs pathology

In [5]:
#plot by disease progression
br1003aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR1003a specs.xlsx',header=10)
br301Specs=pd.read_excel('/media/xinyi/dcis2idc/data/BR301 specs.xlsx',header=10)
br8018aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR8018a specs.xlsx',header=10)
br1003aSpecs.index=br1003aSpecs.loc[:,'Position']
br301Specs.index=br301Specs.loc[:,'Position']
br8018aSpecs.index=br8018aSpecs.loc[:,'Position']

progList=np.copy(allImgNames)
for s in np.unique(allImgNames):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList[allImgNames==s]=prog_s

In [6]:
progNames,progCounts=np.unique(progList,return_counts=True)
progSampleRate={}
for p in range(progNames.size):
    progSampleRate[progNames[p]]=np.min(progCounts)/progCounts[p]
    
plotepoch=311
np.random.seed(6)
plottingIdx_i=np.array([])
n_pcs=50
uniqueImgNames,imgNameIdx=np.unique(allImgNames,return_index=True)
for i in range(1):
    for sidx in range(uniqueImgNames.size):
        s=uniqueImgNames[sidx]
        p=progList[imgNameIdx[sidx]]
        print(s+' '+p)
        nsamples=int(np.sum(allImgNames==s)*progSampleRate[p])
        plottingIdx_i=np.concatenate((plottingIdx_i,
                                    np.random.choice(np.arange(allImgNames.shape[0])[allImgNames==s],nsamples,replace=False)))



plotsavenameAdd='_plottingIdx_progBalanced_'+str(i)


br1003a_1_cytokeratin_555_aSMA_647_hoechst_A1 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A4 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A5 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A6 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A7 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A8 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A9 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C10 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C7 Hyperplasia
br1003a_1_cytokeratin_555_

br301_6_collagen1_647_hoechst_D2 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_D3 Invasive ductal carcinoma and breast tissue
br301_6_collagen1_647_hoechst_D4 Invasive ductal carcinoma
br301_6_collagen1_647_hoechst_D5 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_D6 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E1 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E2 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E3 Invasive ductal carcinoma and breast tissue
br301_6_collagen1_647_hoechst_E4 Invasive ductal carcinoma
br301_6_collagen1_647_hoechst_E5 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E6 Ductal carcinoma in situ with early infiltratio
br8018a_1_cytokeratin_555_aSMA_647_hoechst_A1 Invasive ductal carcinoma
br8018a_1_cytokeratin_555_aSMA_647_hoechst_A10 Invasive ductal carcinoma
br8018a_1_cytoke

In [13]:
nclusterL=[8,12,20,25]
for ncluster in nclusterL:
    savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
        clusterRes=pickle.load(output)
    clusterplotsavepath=os.path.join(clustersavedir,savenamecluster+'_plots')
    if not os.path.exists(clusterplotsavepath):
        os.mkdir(clusterplotsavepath)
    plotCTcomp(clusterRes,progList[plottingIdx_i.astype(int)],clusterplotsavepath,'pathologyHeatmap',False)
    plotCTcomp(clusterRes,progList[plottingIdx_i.astype(int)],clusterplotsavepath,'pathologyHeatmap',True)

### subcluster

In [7]:
for p in np.unique(progList):
    if p=='Ductal carcinoma in situ':
        progList[progList==p]='DCIS'
    elif p=='Ductal carcinoma in situ and breast tissue':
        progList[progList==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ with early infiltratio':
        progList[progList==p]='DCIS with early infiltration'
    elif p=='Micropapillary type ductal carcinoma in situ wi':
        progList[progList==p]='Micropapillary DCIS with early infiltration'


In [10]:
ctorder=np.array(['Breast tissue','Cancer adjacent normal breast tissue','Invasive ductal carcinoma (breast tissue)','Hyperplasia','Atypical hyperplasia','DCIS and breast tissue','DCIS','DCIS with early infiltration', 'Micropapillary DCIS with early infiltration','Invasive ductal carcinoma and breast tissue','Invasive ductal carcinoma'])
# nclusterL=[8,12,20,25]
# subclusterL=[2,3,4,6,8,10,12,15,20]
nclusterL=[8]
subclusterL=[4,6,8]
for ncluster in nclusterL:
    i=0
    savenamecluster0='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster0), 'rb') as output:
        clusterRes=pickle.load(output)
    for i in np.unique(clusterRes):
        subclustersavedir=os.path.join(clustersavedir,savenamecluster0+'_subcluster'+str(i))
        subclusterplotdir=os.path.join(subclustersavedir,'plots')
        
        for subcluster in subclusterL:
            savenamecluster='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
            with open(os.path.join(subclustersavedir,savenamecluster), 'rb') as output:
                subclusterRes=pickle.load(output)
#             plotCTcomp(subclusterRes,progList[plottingIdx_i.astype(int)][clusterRes==i],subclusterplotdir,'pathologyHeatmap_ncluster'+str(subcluster),False)
            plotCTcomp(subclusterRes,progList[plottingIdx_i.astype(int)][clusterRes==i],subclusterplotdir,'pathologyHeatmap_ncluster'+str(subcluster),False,ctorder=ctorder,vmin=0)
        gc.collect()

## cluster vs proteins

### heatmap

In [16]:
stats2plot=['int_median','kurtosis','skewness','int_mode','entropy']
stains=['aSMA','cytokeratin','collagen1','gh2ax']
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

for ncluster in nclusterL:
    savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
        clusterRes=pickle.load(output)
    clusterplotsavepath=os.path.join(clustersavedir,savenamecluster+'_plots')
    if not os.path.exists(clusterplotsavepath):
        os.mkdir(clusterplotsavepath)
        
    allstats=None
    alllabels=None
    allvarnames=None
    for sidx in range(uniquenames.size):
        s=np.unique(allImgNames)[sidx]
        plottingIdx_i_s=plottingIdx_i.astype(int)[allImgNames[plottingIdx_i.astype(int)]==s]-nameIdx[sidx]
        
        print(s)
        assert np.min(plottingIdx_i_s)>=0
#         ssplit=s.split('_')
        slabels=clusterRes[allImgNames[plottingIdx_i.astype(int)]==s]
        for stain_s in stains:
            if not stain_s in s:
                continue
            path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'cellular_'+stain_s.lower(),s.split('_')[-1] +'.csv')
            if not os.path.exists(path_s):
                continue
            print(stain_s)
            stats_s=pd.read_csv(path_s)
            stats_s.index=stats_s.loc[:,'label']
            if allstats is None:
                allstats=stats_s.loc[cellIDlist[s][plottingIdx_i_s],stats2plot].to_numpy()
                alllabels=np.copy(slabels)
                allvarnames=np.repeat(stain_s,slabels.size)
            else:
                allstats=np.concatenate((allstats,stats_s.loc[cellIDlist[s][plottingIdx_i_s],stats2plot].to_numpy()),axis=0)
                alllabels=np.concatenate((alllabels,np.copy(slabels)))
                allvarnames=np.concatenate((allvarnames,np.repeat(stain_s,slabels.size)))
    plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotsavepath,'proteinHeatmap',True)
    plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotsavepath,'proteinHeatmap',False)

            

br1003a_1_cytokeratin_555_aSMA_647_hoechst_A1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A2
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A4
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A5
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A6
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A7
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A8
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A9
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C10
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C2
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C7
aSMA
cyt

br8018a_1_cytokeratin_555_aSMA_647_hoechst_B2
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_B3
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_B4
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_B5
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_B6
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_B7
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_B8
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F1
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F10
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F2
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F3
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F4
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F5
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F6
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F7
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F8
aSMA
cyt

br301_4_cytokeratin_555_aSMA_647_hoechst_A1
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
aSMA
cytokeratin
br301_4_cytokeratin_555_

br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C10
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C2
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C7
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C8
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C9
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I10
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
aSMA
cy

br8018a_1_cytokeratin_555_aSMA_647_hoechst_F10
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F2
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F3
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F4
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F5
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F6
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F7
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F8
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F9
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_H1
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_H2
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_H3
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_H4
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_H5
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_H6
aSMA
cytokeratin
br8018a_1_cytokeratin_555_aSMA_647_hoechst_H7
aSMA
cyt

br301_4_cytokeratin_555_aSMA_647_hoechst_B4
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_C6
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_D1
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_D2
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_D3
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_D4
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_D5
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_D6
aSMA
cytokeratin
br301_4_cytokeratin_555_aSMA_647_hoechst_E1
aSMA
cytokeratin
br301_4_cytokeratin_555_

### subcluster

In [ ]:


# nclusterL=[8,20,12]
# subclusterL=[8,12]

stats2plot=['int_median','kurtosis','skewness','int_mode','entropy']
stains=['aSMA','cytokeratin','collagen1','gh2ax']
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

for ncluster in nclusterL:
    i=0
    savenamecluster0='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster0), 'rb') as output:
        clusterRes=pickle.load(output)
    for i in np.unique(clusterRes):
        if ncluster ==25:
            continue
        print(str(ncluster)+' '+str(i))
        subclustersavedir=os.path.join(clustersavedir,savenamecluster0+'_subcluster'+str(i))
        subclusterplotdir=os.path.join(subclustersavedir,'plots')
        
        imgNames_i=allImgNames[plottingIdx_i.astype(int)][clusterRes==i]
        plottingIdx_i_i=plottingIdx_i.astype(int)[clusterRes==i]
        for subcluster in subclusterL:
            savenamecluster='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
            with open(os.path.join(subclustersavedir,savenamecluster), 'rb') as output:
                subclusterRes=pickle.load(output)
#             plotCTcomp(subclusterRes,progList[plottingIdx_i.astype(int)][clusterRes==i],subclusterplotdir,'pathologyHeatmap_ncluster'+str(subcluster),False)
#             plotCTcomp(subclusterRes,progList[plottingIdx_i.astype(int)][clusterRes==i],subclusterplotdir,'pathologyHeatmap_ncluster'+str(subcluster),True)
            allstats=None
            alllabels=None
            allvarnames=None
            for sidx in range(uniquenames.size):
                s=np.unique(allImgNames)[sidx]
                plottingIdx_i_s=plottingIdx_i_i[imgNames_i==s]-nameIdx[sidx]
                if plottingIdx_i_s.size==0:
                    continue
                print(s)
                assert np.min(plottingIdx_i_s)>=0
        #         ssplit=s.split('_')
                slabels=subclusterRes[imgNames_i==s]
                for stain_s in stains:
                    if not stain_s in s:
                        continue
                    path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'cellular_'+stain_s.lower(),s.split('_')[-1] +'.csv')
                    if not os.path.exists(path_s):
                        continue
                    print(stain_s)
                    stats_s=pd.read_csv(path_s)
                    stats_s.index=stats_s.loc[:,'label']
                    if allstats is None:
                        allstats=stats_s.loc[cellIDlist[s][plottingIdx_i_s],stats2plot].to_numpy()
                        alllabels=np.copy(slabels)
                        allvarnames=np.repeat(stain_s,slabels.size)
                    else:
                        allstats=np.concatenate((allstats,stats_s.loc[cellIDlist[s][plottingIdx_i_s],stats2plot].to_numpy()),axis=0)
                        alllabels=np.concatenate((alllabels,np.copy(slabels)))
                        allvarnames=np.concatenate((allvarnames,np.repeat(stain_s,slabels.size)))
            plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,subclusterplotdir,'proteinHeatmap_ncluster'+str(subcluster),True)
            gc.collect()

### duct membership histogram
subdivide cluster by pathology

In [12]:
nclusterL=[8,12,20,25]
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

for ncluster in nclusterL:
    savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
        clusterRes=pickle.load(output)
    clusterplotsavepath=os.path.join(clustersavedir,savenamecluster+'_plots')
    if not os.path.exists(clusterplotsavepath):
        os.mkdir(clusterplotsavepath)
        
    allstats=None
    alllabels=None
    allvarnames=None
    for sidx in range(uniquenames.size):
        s=np.unique(allImgNames)[sidx]
        
        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(s)
        plottingIdx_i_s=plottingIdx_i.astype(int)[allImgNames[plottingIdx_i.astype(int)]==s]-nameIdx[sidx]
        assert np.min(plottingIdx_i_s)>=0
        
        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s],'int_median'].to_numpy()

        
#         ssplit=s.split('_')
        slabels=clusterRes[allImgNames[plottingIdx_i.astype(int)]==s]
    
        if allstats is None:
            allstats=stats_s>0
            alllabels=np.copy(slabels)
        else:
            allstats=np.concatenate((allstats,stats_s>0))
            alllabels=np.concatenate((alllabels,np.copy(slabels)))
    plotCTcomp_hist(alllabels,allstats,clusterplotsavepath,'ductHist',True)
            

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

In [13]:
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

for ncluster in nclusterL:
    savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
        clusterRes=pickle.load(output)
    clusterplotsavepath=os.path.join(clustersavedir,savenamecluster+'_plots')
    if not os.path.exists(clusterplotsavepath):
        os.mkdir(clusterplotsavepath)
        
    allstats=None
    alllabels=None
    allprogs=None
    for sidx in range(uniquenames.size):
        s=np.unique(allImgNames)[sidx]
        
        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(s)
        plottingIdx_i_s=plottingIdx_i.astype(int)[allImgNames[plottingIdx_i.astype(int)]==s]-nameIdx[sidx]
        assert np.min(plottingIdx_i_s)>=0
        
        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s],'int_median'].to_numpy()

        
#         ssplit=s.split('_')
        slabels=clusterRes[allImgNames[plottingIdx_i.astype(int)]==s]
        sprog=progList[plottingIdx_i.astype(int)][allImgNames[plottingIdx_i.astype(int)]==s]
    
        if allstats is None:
            allstats=stats_s>0
            alllabels=np.copy(slabels)
            allprogs=np.copy(sprog)
        else:
            allstats=np.concatenate((allstats,stats_s>0))
            alllabels=np.concatenate((alllabels,np.copy(slabels)))
            allprogs=np.concatenate((allprogs,np.copy(sprog)))
    for p in np.unique(allprogs):
        plotCTcomp_hist(alllabels[allprogs==p],allstats[allprogs==p],clusterplotsavepath,'ductHist'+p,True)
            

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

### subcluster

In [19]:
subclusterplotdir

'/media/xinyi/dcis2idc/plots/cnnvaeexp0/cluster/minibatchkmean_ncluster12n_pcs50epoch311_plottingIdx_progBalanced_0_subcluster5/plots'

In [15]:

# nclusterL=[8,20,12]
subclusterL=[4,6,8]

uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

for ncluster in nclusterL:
    if ncluster ==25:
        continue
    i=0
    savenamecluster0='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster0), 'rb') as output:
        clusterRes=pickle.load(output)
    for i in np.unique(clusterRes):
        subclustersavedir=os.path.join(clustersavedir,savenamecluster0+'_subcluster'+str(i))
        subclusterplotdir=os.path.join(subclustersavedir,'plots')
        
        imgNames_i=allImgNames[plottingIdx_i.astype(int)][clusterRes==i]
        plottingIdx_i_i=plottingIdx_i.astype(int)[clusterRes==i]
        for subcluster in subclusterL:
            savenamecluster='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
            with open(os.path.join(subclustersavedir,savenamecluster), 'rb') as output:
                subclusterRes=pickle.load(output)
            
            allstats=None
            alllabels=None
            allprogs=None
            for sidx in range(uniquenames.size):
                s=np.unique(allImgNames)[sidx]

                path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
                if not os.path.exists(path_s):
                    continue
                print(s)
                plottingIdx_i_s=plottingIdx_i_i[imgNames_i==s]-nameIdx[sidx]
                if plottingIdx_i_s.size==0:
                    continue
                assert np.min(plottingIdx_i_s)>=0

                stats_s=pd.read_csv(path_s)
                stats_s.index=stats_s.loc[:,'label']
                stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s],'int_median'].to_numpy()


        #         ssplit=s.split('_')
                slabels=subclusterRes[imgNames_i==s]
                sprog=progList[plottingIdx_i_i][imgNames_i==s]

                if allstats is None:
                    allstats=stats_s>0
                    alllabels=np.copy(slabels)
                    allprogs=np.copy(sprog)
                else:
                    allstats=np.concatenate((allstats,stats_s>0))
                    alllabels=np.concatenate((alllabels,np.copy(slabels)))
                    allprogs=np.concatenate((allprogs,np.copy(sprog)))
            for p in np.unique(allprogs):
                plotCTcomp_hist(alllabels[allprogs==p],allstats[allprogs==p],subclusterplotdir,'ductHist'+p+'_ncluster'+str(subcluster),True)

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

KeyboardInterrupt: 

In [15]:
#by samples
nclusterL=[8]
subclusterL=[4,6,8]

uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

for ncluster in nclusterL:
    savenamecluster0='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster0), 'rb') as output:
        clusterRes=pickle.load(output)
    for i in np.unique(clusterRes):
        subclustersavedir=os.path.join(clustersavedir,savenamecluster0+'_subcluster'+str(i))
        subclusterplotdir=os.path.join(subclustersavedir,'plots')
        
        imgNames_i=allImgNames[plottingIdx_i.astype(int)][clusterRes==i]
        plottingIdx_i_i=plottingIdx_i.astype(int)[clusterRes==i]
        for subcluster in subclusterL:
            savenamecluster='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
            with open(os.path.join(subclustersavedir,savenamecluster), 'rb') as output:
                subclusterRes=pickle.load(output)
            
            allstats=None
            alllabels=None
            allprogs=None
            allnames=None
            for sidx in range(uniquenames.size):
                s=np.unique(allImgNames)[sidx]

                path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
                if not os.path.exists(path_s):
                    continue
                print(s)
                plottingIdx_i_s=plottingIdx_i_i[imgNames_i==s]-nameIdx[sidx]
                if plottingIdx_i_s.size==0:
                    continue
                assert np.min(plottingIdx_i_s)>=0

                stats_s=pd.read_csv(path_s)
                stats_s.index=stats_s.loc[:,'label']
                stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s],'int_median'].to_numpy()


        #         ssplit=s.split('_')
                slabels=subclusterRes[imgNames_i==s]
                sprog=progList[plottingIdx_i_i][imgNames_i==s]

                if allstats is None:
                    allstats=stats_s>0
                    alllabels=np.copy(slabels)
                    allprogs=np.copy(sprog)
                    allnames=np.repeat(s,stats_s.size)
                else:
                    allstats=np.concatenate((allstats,stats_s>0))
                    alllabels=np.concatenate((alllabels,np.copy(slabels)))
                    allprogs=np.concatenate((allprogs,np.copy(sprog)))
                    allnames=np.concatenate((allnames,np.repeat(s,stats_s.size)))
            for p in np.unique(allprogs):
                plotCTcomp_hist_cont(alllabels[allprogs==p],allstats[allprogs==p],allnames[allprogs==p],subclusterplotdir,'ductHistBySamples'+p+'_ncluster'+str(subcluster))

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_555_aSMA_647_hoechst_C6
br301_4_cytokeratin_555_aSMA_647_hoechst_D1
br301_4_cytokeratin_555_aSMA_647_hoechst_D2
br301_4_cytokeratin_555_aSMA_647_hoechst_D3
br301_4_cytokeratin_555_aSMA_647_hoechst_D4
br301_4_cytokeratin_555_aSMA_647_hoechst_D5
br301_4_cytokeratin_555_aSMA_647_hoechst_D6
br301_4_cytokeratin_555_aSMA_647_hoechst_E1
br301_4_cytokeratin_555_aSMA_647_hoechst_E2
br301_4_cytokeratin_555_aSMA_647_hoechst_E3
br301_4_cytokeratin_555_aSMA_647_hoechst_E4
br301_4_cytokeratin_555_aSMA_647_hoechst_E5
br301_4_cytokeratin_555_aSMA_647_hoechst_E6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F2
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_

br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_555_aSMA_647_hoechst_C6
br301_4_cytokeratin_555_aSMA_647_hoechst_D1
br301_4_cytokeratin_555_aSMA_647_hoechst_D2
br301_4_cytokeratin_555_aSMA_647_hoechst_D3
br301_4_cytokeratin_555_aSMA_647_hoechst_D4
br301_4_cytokeratin_555_aSMA_647_hoechst_D5
br301_4_cytokeratin_555_aSMA_647_hoechst_D6
br301_4_cytokeratin_555_aSMA_647_hoechst_E1
br301_4_cytokeratin_555_aSMA_647_hoechst_E2
br301_4_cytokeratin_555_aSMA_647_hoechst_E3
br301_4_cytokeratin_555_aSMA_647_hoechst_E4
br301_4_cytokeratin_555_aSMA_647_hoechst_E5
br301_4_cytokeratin_555_aSMA_647_hoechst_E6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F2
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_

br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_555_aSMA_647_hoechst_C6
br301_4_cytokeratin_555_aSMA_647_hoechst_D1
br301_4_cytokeratin_555_aSMA_647_hoechst_D2
br301_4_cytokeratin_555_aSMA_647_hoechst_D3
br301_4_cytokeratin_555_aSMA_647_hoechst_D4
br301_4_cytokeratin_555_aSMA_647_hoechst_D5
br301_4_cytokeratin_555_aSMA_647_hoechst_D6
br301_4_cytokeratin_555_aSMA_647_hoechst_E1
br301_4_cytokeratin_555_aSMA_647_hoechst_E2
br301_4_cytokeratin_555_aSMA_647_hoechst_E3
br301_4_cytokeratin_555_aSMA_647_hoechst_E4
br301_4_cytokeratin_555_aSMA_647_hoechst_E5
br301_4_cytokeratin_555_aSMA_647_hoechst_E6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F2
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_

In [15]:
#plot in sample
maxsample=15

subclusternumbers=[4,6,8,6,6,6,6,4]

stains=['aSMA','cytokeratin','collagen1','gh2ax']
dnaRGB=(0,191/255,1)
protein1RGB=(1,1,1)
protein2RGB=(1,215/255,0)
ductRGB=(199/255,21/255,133/255)
boxRGB=(1,0,0)
boxThickness=10
boxWidth=100

ncluster=8

uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

savenamecluster0='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
with open(os.path.join(clustersavedir,savenamecluster0), 'rb') as output:
    clusterRes=pickle.load(output)
for i in np.unique(clusterRes):
    subclustersavedir=os.path.join(clustersavedir,savenamecluster0+'_subcluster'+str(i))
    subclusterplotdir=os.path.join(subclustersavedir,'plots','ductsComposite')
    if not os.path.exists(subclusterplotdir):
        os.mkdir(subclusterplotdir)

    imgNames_i=allImgNames[plottingIdx_i.astype(int)][clusterRes==i]
    plottingIdx_i_i=plottingIdx_i.astype(int)[clusterRes==i]
    subcluster=subclusternumbers[i]
    savenamecluster='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(subclustersavedir,savenamecluster), 'rb') as output:
        subclusterRes=pickle.load(output)

    allstats=None
    alllabels=None
    allprogs=None
    allnames=None
    for sidx in range(uniquenames.size):
        s=np.unique(allImgNames)[sidx]

        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(s)
        plottingIdx_i_s=plottingIdx_i_i[imgNames_i==s]-nameIdx[sidx]
        if plottingIdx_i_s.size==0:
            continue
        assert np.min(plottingIdx_i_s)>=0

        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s],'int_median'].to_numpy()


#         ssplit=s.split('_')
        slabels=subclusterRes[imgNames_i==s]
        sprog=progList[plottingIdx_i_i][imgNames_i==s]


        scoord=coordlist[plottingIdx_i_i][imgNames_i==s]

        ductSeg=io.imread(os.path.join(datadir,'_'.join(s.split('_')[:-1]),'ducts_segmented_interactive',s.split('_')[-1] +'.tif'))
        ductSeg=np.where(ductSeg>0,0.3,0)
        zprojectDNA=io.imread(os.path.join(datadir,'_'.join(s.split('_')[:-1]),'zproject','dna',s.split('_')[-1] +'.tif'))
        zprojectDNA=(zprojectDNA-np.min(zprojectDNA))/(np.max(zprojectDNA)-np.min(zprojectDNA))
        protein1=None
        protein2=None
        for st in stains:
            if os.path.exists(os.path.join(datadir,'_'.join(s.split('_')[:-1]),'zproject',st)):
                if protein1 is None:
                    protein1=io.imread(os.path.join(datadir,'_'.join(s.split('_')[:-1]),'zproject',st,s.split('_')[-1] +'.tif'))
                    protein1=(protein1-np.min(protein1))/(np.max(protein1)-np.min(protein1))
                else:
                    protein2=io.imread(os.path.join(datadir,'_'.join(s.split('_')[:-1]),'zproject',st,s.split('_')[-1] +'.tif'))
                    protein2=(protein2-np.min(protein2))/(np.max(protein2)-np.min(protein2))

        zprojectDNA=zprojectDNA[:,:,np.newaxis]*np.asarray(dnaRGB).reshape((1, 1, -1))
        ductSeg=ductSeg[:,:,np.newaxis]*np.asarray(ductRGB).reshape((1, 1, -1))
        protein1=protein1[:,:,np.newaxis]*np.asarray(protein1RGB).reshape((1, 1, -1))
        if protein2 is not None:
            protein2=protein2[:,:,np.newaxis]*np.asarray(protein2RGB).reshape((1, 1, -1))

        img=zprojectDNA+ductSeg+protein1
        if protein2 is not None:
            img+=protein2
        img=(img-np.min(img))/(np.max(img)-np.min(img))

        for plotsub in np.unique(slabels):
            sampled=[]
            induct=stats_s>0
            outduct=stats_s==0
            sub_c=slabels==plotsub
            
            inIdx=np.arange(slabels.size)[np.logical_and(induct,sub_c)]
            outIdx=np.arange(slabels.size)[np.logical_and(outduct,sub_c)]
            np.random.seed(3)
            inIdx_sample=np.random.choice(inIdx,min(maxsample,inIdx.size),replace=False)
            np.random.seed(3)
            outIdx_sample=np.random.choice(outIdx,min(maxsample,outIdx.size),replace=False)
            for incell in inIdx_sample:
                coord_c=scoord[incell]
                img=cv2.rectangle(img,(int(coord_c[1]-boxWidth/2),int(coord_c[0]-boxWidth/2)),(int(coord_c[1]+boxWidth/2),int(coord_c[0]+boxWidth/2)),boxRGB,boxThickness)
            
            for incell in outIdx_sample:
                coord_c=scoord[incell]
                img=cv2.rectangle(img,(int(coord_c[1]-boxWidth/2),int(coord_c[0]-boxWidth/2)),(int(coord_c[1]+boxWidth/2),int(coord_c[0]+boxWidth/2)),boxRGB,boxThickness)

            plt.imsave(os.path.join(subclusterplotdir,s+'_'+str(plotsub)+'.jpg'),img)

                
#                 if allstats is None:
#                     allstats=stats_s>0
#                     alllabels=np.copy(slabels)
#                     allprogs=np.copy(sprog)
#                     allnames=np.repeat(s,stats_s.size)
#                 else:
#                     allstats=np.concatenate((allstats,stats_s>0))
#                     alllabels=np.concatenate((alllabels,np.copy(slabels)))
#                     allprogs=np.concatenate((allprogs,np.copy(sprog)))
#                     allnames=np.concatenate((allnames,np.repeat(s,stats_s.size)))
#             for p in np.unique(allprogs):
#                 plotCTcomp_hist_cont(alllabels[allprogs==p],allstats[allprogs==p],allnames[allprogs==p],subclusterplotdir,'ductHistBySamples'+p+'_ncluster'+str(subcluster))

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_555_aSMA_647_hoechst_C6
br301_4_cytokeratin_555_aSMA_647_hoechst_D1
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_555_aSMA_647_hoechst_C6
br301_4_cytokeratin_555_aSMA_647_hoechst_D1
br301_4_cytokeratin_555_aS

KeyboardInterrupt: 

In [45]:
#distance to nearest cell in duct
maxsample=15

subclusternumbers=[4,6,8,6,6,6,6,4]

ncluster=8

uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

savenamecluster0='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
with open(os.path.join(clustersavedir,savenamecluster0), 'rb') as output:
    clusterRes=pickle.load(output)
for i in np.unique(clusterRes):
    subclustersavedir=os.path.join(clustersavedir,savenamecluster0+'_subcluster'+str(i))
    subclusterplotdir=os.path.join(subclustersavedir,'plots')
    if not os.path.exists(subclusterplotdir):
        os.mkdir(subclusterplotdir)

    imgNames_i=allImgNames[plottingIdx_i.astype(int)][clusterRes==i]
    plottingIdx_i_i=plottingIdx_i.astype(int)[clusterRes==i]
    subcluster=subclusternumbers[i]
    savenamecluster='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(subclustersavedir,savenamecluster), 'rb') as output:
        subclusterRes=pickle.load(output)

   
    distances=np.array([])
    sublabels=np.array([])
    subProgs=np.array([])
    for sidx in range(uniquenames.size):
        s=np.unique(allImgNames)[sidx]

        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(s)
        plottingIdx_i_s=plottingIdx_i_i[imgNames_i==s]-nameIdx[sidx]
        if plottingIdx_i_s.size==0:
            continue
        assert np.min(plottingIdx_i_s)>=0

        stats_s_all=pd.read_csv(path_s)
        stats_s_all.index=stats_s_all.loc[:,'label']
        stats_s=stats_s_all.loc[cellIDlist[s][plottingIdx_i_s],'int_median'].to_numpy()

        stats_s_all=stats_s_all.loc[cellIDlist[s],'int_median'].to_numpy()
        scoord_all_duct=coordlist[allImgNames==s][stats_s_all>0]

#         ssplit=s.split('_')
        slabels=subclusterRes[imgNames_i==s]
        sprog=progList[plottingIdx_i_i][imgNames_i==s]


        scoord=coordlist[plottingIdx_i_i][imgNames_i==s][stats_s==0]
        
        _,distS=sklearn.metrics.pairwise_distances_argmin_min(scoord,scoord_all_duct)
        distances=np.concatenate((distances,distS))
        sublabels=np.concatenate((sublabels,slabels[stats_s==0]))
        subProgs=np.concatenate((subProgs,sprog[stats_s==0]))
     
    fig, ax = plt.subplots(nrows=np.unique(sublabels).size, ncols=1,figsize=(10, 10),sharex=True,sharey=True)
    for r in range(np.unique(sublabels).size):
        slabels_r=np.unique(sublabels)[r]
        res=pd.DataFrame.from_dict({'distances':distances[sublabels==slabels_r],'disease':subProgs[sublabels==slabels_r]})
        
        sns.violinplot(x="disease", y="distances",cut=0,
                    data=res, palette="muted", legend=False,order=np.unique(res['disease']),ax=ax[r])
        ax[r].set_xticks(np.arange(np.unique(res['disease']).size))
        ax[r].set_xticklabels(np.unique(res['disease']))
        ax[r].set_ylabel(slabels_r)
        ax[r].set_xlabel('')
    plt.xticks(rotation=90)
    fig.tight_layout()
    plt.savefig(os.path.join(subclusterplotdir,'distance2Duct.jpg'))
    plt.close()
        
            
                
#                 if allstats is None:
#                     allstats=stats_s>0
#                     alllabels=np.copy(slabels)
#                     allprogs=np.copy(sprog)
#                     allnames=np.repeat(s,stats_s.size)
#                 else:
#                     allstats=np.concatenate((allstats,stats_s>0))
#                     alllabels=np.concatenate((alllabels,np.copy(slabels)))
#                     allprogs=np.concatenate((allprogs,np.copy(sprog)))
#                     allnames=np.concatenate((allnames,np.repeat(s,stats_s.size)))
#             for p in np.unique(allprogs):
#                 plotCTcomp_hist_cont(alllabels[allprogs==p],allstats[allprogs==p],allnames[allprogs==p],subclusterplotdir,'ductHistBySamples'+p+'_ncluster'+str(subcluster))

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br301_4_cytokeratin_555_aSMA_647_hoechst_A1
br301_4_cytokeratin_555_aSMA_647_hoechst_A2
br301_4_cytokeratin_555_aSMA_647_hoechst_A3
br301_4_cytokeratin_555_aSMA_647_hoechst_A4
br301_4_cytokeratin_555_aSMA_647_hoechst_A5
br301_4_cytokeratin_555_aSMA_647_hoechst_A6
br301_4_cytokeratin_555_aSMA_647_hoechst_B1
br301_4_cytokeratin_555_aSMA_647_hoechst_B2
br301_4_cytokeratin_555_aSMA_647_hoechst_B3
br301_4_cytokeratin_555_aSMA_647_hoechst_B4
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_C1
br301_4_cytokeratin_555_aSMA_647_hoechst_C2
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_4_cytokeratin_555_aSMA_647_hoechst_C5
br301_4_cytokeratin_55

### plot nuclei in each clusters

In [19]:
with open(os.path.join(datadir,'processed','train_cnnvae'), 'rb') as input:
    allImg=pickle.load(input)

In [20]:
#Sample 10 nuclei/pathology from each cluster
ncells=20

for ncluster in nclusterL:
    savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
        clusterRes=pickle.load(output)
    clusterplotsavepath=os.path.join(clustersavedir,savenamecluster+'_plots')
    if not os.path.exists(clusterplotsavepath):
        os.mkdir(clusterplotsavepath)
        
    progs=np.unique(progList[plottingIdx_i.astype(int)])
    for p in range(progs.size):
        img_p=allImg[plottingIdx_i.astype(int)][progList[plottingIdx_i.astype(int)]==progs[p]]
        cluster_p=clusterRes[progList[plottingIdx_i.astype(int)]==progs[p]]
        plotExample=np.zeros((np.unique(cluster_p).size*(radius+1)*2,ncells*(radius+1)*2))
        for c in range(np.unique(cluster_p).size):
            clusteridx=np.arange(img_p.shape[0])[cluster_p==np.unique(cluster_p)[c]]
            np.random.seed(6)
            cp_plot=np.random.choice(clusteridx,min(ncells,clusteridx.size),replace=False)
            for cp_i in range(cp_plot.size):
                plotExample[c*(radius*2+2):c*(radius*2+2)+radius*2,cp_i*(radius*2+2):cp_i*(radius*2+2)+radius*2]=img_p[cp_plot[cp_i]]
        fig, ax = plt.subplots(figsize=(20,20))
        img = ax.imshow(plotExample)
        ax.set_yticks(np.arange(np.unique(cluster_p).size).astype(int)*(radius*2+2)+(radius+1))
        ax.set_yticklabels(np.unique(cluster_p))
        fig.tight_layout()
        plt.savefig(os.path.join(clusterplotsavepath,'plotexamples'+progs[p]+'.jpg'))
        plt.close()

In [21]:
allImg=None
gc.collect()

161061

#### plot patch around cells in each cell clusters

In [ ]:
#RUN
with open(os.path.join(datadir,'processed','train_cnnvae_patch'+'_plottingIdx_'+str(plotPCT)+'_'+str(i)+'flt32'), 'rb') as input:
    allImg_patch=pickle.load(input)

In [16]:
#Sample 10 nuclei/pathology from each cluster
ncells=15
nclusterL=[8,12,20]
patchsize=16*9

for ncluster in nclusterL:
    savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
        clusterRes=pickle.load(output)
    clusterplotsavepath=os.path.join(clustersavedir,savenamecluster+'_plots')
        
    progs=np.unique(progList[plottingIdx_i.astype(int)])
    for p in range(progs.size):
        if os.path.exists(os.path.join(clusterplotsavepath,'plotexamples'+progs[p]+'_patch.jpg')):
            continue
        img_p=allImg_patch[progList[plottingIdx_i.astype(int)]==progs[p]]
        cluster_p=clusterRes[progList[plottingIdx_i.astype(int)]==progs[p]]
        plotExample=np.zeros((np.unique(cluster_p).size*(patchsize+1)*2,ncells*(patchsize+1)*2))
        for c in range(np.unique(cluster_p).size):
            clusteridx=np.arange(img_p.shape[0])[cluster_p==np.unique(cluster_p)[c]]
            np.random.seed(6)
            cp_plot=np.random.choice(clusteridx,min(ncells,clusteridx.size),replace=False)
            for cp_i in range(cp_plot.size):
                plotExample[c*(patchsize*2+2):c*(patchsize*2+2)+patchsize*2,cp_i*(patchsize*2+2):cp_i*(patchsize*2+2)+patchsize*2]=img_p[cp_plot[cp_i]]
        fig, ax = plt.subplots(figsize=(20,20))
        img = ax.imshow(plotExample)
        ax.set_yticks(np.arange(np.unique(cluster_p).size).astype(int)*(patchsize*2+2)+(patchsize+1))
        ax.set_yticklabels(np.unique(cluster_p))
        fig.tight_layout()
        plt.savefig(os.path.join(clusterplotsavepath,'plotexamples'+progs[p]+'_patch.jpg'))
        plt.close()
        gc.collect()

In [14]:
ncluster

8

## cluster vs background cluster

In [12]:
sampledir_patch='/media/xinyi/dcis2idc/plots/cnnvaeexp0_background'
clustersavedir_patch=os.path.join(sampledir_patch,'cluster')
patchEpoch=76
for ncluster in [8]:
    savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
        clusterRes=pickle.load(output)
    clusterplotsavepath=os.path.join(clustersavedir,savenamecluster+'_plots')
    if not os.path.exists(clusterplotsavepath):
        os.mkdir(clusterplotsavepath)
    
    savenamecluster_patch='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(patchEpoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir_patch,savenamecluster_patch), 'rb') as output:
        clusterRes_patch=pickle.load(output)
       
    plotCTcomp(clusterRes,clusterRes_patch,clusterplotsavepath,'bgHeatmap',False)
    plotCTcomp(clusterRes,clusterRes_patch,clusterplotsavepath,'bgHeatmap',True)

In [16]:
np.unique(allImgNames).size

215

## neighbor cluster ID

In [13]:
from sklearn.metrics import pairwise_distances

### distance threshold

In [44]:
# maxncell=130000

neighborhoodSize=16*9
#neighborhoodSize=16*9*1.5 #run later
for ncluster in [8,12,20]:
    savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+plotsavenameAdd
    with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
        clusterRes=pickle.load(output)
    clusterplotsavepath=os.path.join(clustersavedir,savenamecluster+'_plots')
    if not os.path.exists(clusterplotsavepath):
        os.mkdir(clusterplotsavepath)
    progs=np.unique(progList[plottingIdx_i.astype(int)])
    for p in range(progs.size):
        print(p)
        coord_p=coordlist[plottingIdx_i.astype(int)][progList[plottingIdx_i.astype(int)]==progs[p]]
        cluster_p=clusterRes[progList[plottingIdx_i.astype(int)]==progs[p]]
        imgNames_p=allImgNames[plottingIdx_i.astype(int)][progList[plottingIdx_i.astype(int)]==progs[p]]
        
        selfp=np.array([])
        neighborp=np.array([])
        for imgN in np.unique(imgNames_p):
            print(imgN)
#         subsampleIdx=np.arange(coord_p.shape[0])
#         if coord_p.shape[0]>maxncell:
#             np.random.seed(6)
#             subsampleIdx=np.random.choice(subsampleIdx,maxncell,replace=False)
        
            neighbors=np.tile(cluster_p[imgNames_p==imgN],(np.sum(imgNames_p==imgN),1))
            selfs=np.tile(cluster_p[imgNames_p==imgN].reshape((-1,1)),(1,np.sum(imgNames_p==imgN)))
            dist=pairwise_distances(coord_p[imgNames_p==imgN],n_jobs=-1)
            distIn=np.logical_and(dist<neighborhoodSize,dist>0)
            selfp=np.concatenate((selfp,selfs[distIn]))
            neighborp=np.concatenate((neighborp,neighbors[distIn]))
            
        plotCTcomp_hist(selfp,neighborp,clusterplotsavepath,'neighborHist'+progs[p],False)
        gc.collect()

0
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6
br1003a_3_collagen1_647_hoechst_C1
br1003a_3_collagen1_647_hoechst_C3
br1003a_3_collagen1_647_hoechst_C4
br1003a_3_collagen1_647_hoechst_C5
br1003a_3_collagen1_647_hoechst_C6
br1003a_4_cytokeratin_555_gh2ax_647_hoechst_C1
br1003a_4_cytokeratin_555_gh2ax_647_hoechst_C4
br1003a_4_cytokeratin_555_gh2ax_647_hoechst_C5
br1003a_4_cytokeratin_555_gh2ax_647_hoechst_C6
1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I10
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I9
br1003a_3_collagen1_647_hoechst_I1
br1003a_3_collagen1_647_hoechst_I10
br1003a_3

br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F4
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F7
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F8
8
br8018a_1_cytokeratin_555_aSMA_647_hoechst_B6
br8018a_3_collagen1_647_hoechst_B6
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B6
9
br301_4_cytokeratin_555_aSMA_647_hoechst_B5
br301_4_cytokeratin_555_aSMA_647_hoechst_B6
br301_4_cytokeratin_555_aSMA_647_hoechst_D3
br301_4_cytokeratin_555_aSMA_647_hoechst_E3
br301_6_collagen1_647_hoechst_B5
br301_6_collagen1_647_hoechst_B6
br301_6_collagen1_647_hoechst_D3
br301_6_collagen1_647_hoechst_E3
10
br301_4_cytokeratin_555_aSMA_647_hoechst_C3
br301_4_cytokeratin_555_aSMA_647_hoechst_C4
br301_6_collagen1_647_hoechst_C3
br301_6_collagen1_647_hoechst_C4
0
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6
br1003a_3_collagen1_647_hoechs

br8018a_3_collagen1_647_hoechst_B7
br8018a_3_collagen1_647_hoechst_B8
br8018a_3_collagen1_647_hoechst_B9
br8018a_3_collagen1_647_hoechst_F10
br8018a_3_collagen1_647_hoechst_F3
br8018a_3_collagen1_647_hoechst_F4
br8018a_3_collagen1_647_hoechst_F6
br8018a_3_collagen1_647_hoechst_F7
br8018a_3_collagen1_647_hoechst_F8
br8018a_3_collagen1_647_hoechst_F9
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A2
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A3
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A5
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A7
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A8
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B4
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B5
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B7
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F3
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F4
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F7
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F8
8
br8018a_1_cytokeratin_555_aSMA_647_hoechst_B6
br8018a_3_collagen1_647_hoechst_B6
br

br8018a_1_cytokeratin_555_aSMA_647_hoechst_F4
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F5
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F7
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F8
br8018a_1_cytokeratin_555_aSMA_647_hoechst_F9
br8018a_3_collagen1_647_hoechst_A1
br8018a_3_collagen1_647_hoechst_A2
br8018a_3_collagen1_647_hoechst_A3
br8018a_3_collagen1_647_hoechst_A4
br8018a_3_collagen1_647_hoechst_A6
br8018a_3_collagen1_647_hoechst_A7
br8018a_3_collagen1_647_hoechst_A8
br8018a_3_collagen1_647_hoechst_A9
br8018a_3_collagen1_647_hoechst_B1
br8018a_3_collagen1_647_hoechst_B10
br8018a_3_collagen1_647_hoechst_B2
br8018a_3_collagen1_647_hoechst_B3
br8018a_3_collagen1_647_hoechst_B4
br8018a_3_collagen1_647_hoechst_B5
br8018a_3_collagen1_647_hoechst_B7
br8018a_3_collagen1_647_hoechst_B8
br8018a_3_collagen1_647_hoechst_B9
br8018a_3_collagen1_647_hoechst_F10
br8018a_3_collagen1_647_hoechst_F3
br8018a_3_collagen1_647_hoechst_F4
br8018a_3_collagen1_64

## plot subcluster umap with coord of full umap

In [11]:
ep=311
n_neighbors=10
min_dist=0.25
savenameAdd='_nn'+str(n_neighbors)+'mdist0'+str(int(min_dist*100))+'epoch'+str(ep)

with open(os.path.join(modelsavepath,'umap_plottingIdx_'+str(plotPCT)+'_'+str(i)+savenameAdd), 'rb') as output:
    reducer=pickle.load(output)

In [12]:
dim1=0
dim2=1
nclusterL=[8,12,20]
subclusterL=[8,12]
np.random.seed(6)
plotsubsampleRatio=0.5
plottingIdx_i_subsampleIdx=np.random.choice(np.arange(plottingIdx_i.size),int(plottingIdx_i.size*plotsubsampleRatio),replace=False)
plottingIdx_i_subsample=plottingIdx_i[plottingIdx_i_subsampleIdx]
plottingIdx_i_subsampleBool=np.repeat(False,plottingIdx_i.shape)
plottingIdx_i_subsampleBool[plottingIdx_i_subsampleIdx.astype(int)]=True
for ncluster in nclusterL:
    savenamecluster0='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(ep)+'_plottingIdx_'+str(plotPCT)+'_'+str(0)
    with open(os.path.join(clustersavedir,savenamecluster0), 'rb') as output:
        clusterRes=pickle.load(output)
    for c in np.unique(clusterRes):
        print(c)
        subclustersavedir=os.path.join(clustersavedir,savenamecluster0+'_subcluster'+str(c))
        subclusterplotdir=os.path.join(subclustersavedir,'plots')    
#         plotembeddingbyCT(progList[plottingIdx_i.astype(int)][clusterRes==c],'prog_umaptrain_plottingIdx_'+str(plotPCT)+'_'+str(i),[],reduceri.embedding_,subclusterplotdir,plottype+' of all samples',plotdimx=dim1,plotdimy=dim2,savenameAdd=savenameAdd+'_pc'+str(dim1)+'pc'+str(dim2),ncol=2)
#         plotembeddingbyCT_contrast(progList[plottingIdx_i.astype(int)][clusterRes==c],'prog_umaptrain_plottingIdx_'+str(plotPCT)+'_'+str(i),[],reduceri.embedding_,os.path.join(subclusterplotdir,'contrast'),plottype+' of all samples',plotdimx=dim1,plotdimy=dim2,savenameAdd=savenameAdd+'_pc'+str(dim1)+'pc'+str(dim2))
        
        for subcluster in subclusterL:
            savenamecluster='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(ep)+'_plottingIdx_'+str(plotPCT)+'_'+str(c)
            with open(os.path.join(subclustersavedir,savenamecluster), 'rb') as output:
                subclusterRes=pickle.load(output)
            plotembeddingbyCT(subclusterRes[plottingIdx_i_subsampleBool[clusterRes==c]],'fullumapcoord_umaptrain_plottingIdx_'+str(plotPCT)+'_'+str(i)+'_ncluster'+str(subcluster),[],reducer.embedding_[clusterRes[plottingIdx_i_subsampleIdx.astype(int)]==c],subclusterplotdir,plottype+' of all samples',plotdimx=dim1,plotdimy=dim2,savenameAdd=savenameAdd+'_pc'+str(dim1)+'pc'+str(dim2),ncol=2)
            plotembeddingbyCT_contrast(subclusterRes[plottingIdx_i_subsampleBool[clusterRes==c]],'fullumapcoord_umaptrain_plottingIdx_'+str(plotPCT)+'_'+str(i)+'_ncluster'+str(subcluster),[],reducer.embedding_[clusterRes[plottingIdx_i_subsampleIdx.astype(int)]==c],os.path.join(subclusterplotdir,'contrast'),plottype+' of all samples',plotdimx=dim1,plotdimy=dim2,savenameAdd=savenameAdd+'_pc'+str(dim1)+'pc'+str(dim2))
            gc.collect()

0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
14
15
16
17
18
19
